## Part 2: Web scraping data of population and demographic data for NYC from Wikipedia

#### POPULATION DATA
Web scrapping of Population data from wikipedia page - https://en.wikipedia.org/wiki/New_York_City

First, we shall download all the packages as needed.

In [14]:
!pip install geopy
!pip install folium
import folium
import numpy as np
import pandas as pd
import requests
import json
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt

from pandas.io.json import json_normalize
from geopy.geocoders import Nominatim

from bs4 import BeautifulSoup

import csv

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

print('Libraries imported')

Libraries imported


Web scraping the data from Wikipedia using Beautiful Soup

In [15]:
website_url = requests.get('https://en.wikipedia.org/wiki/Demographics_of_New_York_City').text
soup = BeautifulSoup(website_url,'lxml')
table = soup.find('table',{'class':'wikitable sortable'})
#print(soup.prettify())

headers = [header.text for header in table.find_all('th')]

table_rows = table.find_all('tr')        
rows = []
for row in table_rows:
   td = row.find_all('td')
   row = [row.text for row in td]
   rows.append(row)

with open('BON2_POPULATION1.csv', 'w') as f:
   writer = csv.writer(f)
   writer.writerow(headers)
   writer.writerows(row for row in rows if row)

Load data from CSV

In [16]:
Pop_data=pd.read_csv('BON2_POPULATION1.csv')
Pop_data.drop(Pop_data.columns[[7,8,9,10,11]], axis=1,inplace=True)
print('Data downloaded!')

Data downloaded!


Removing white spaces

In [17]:
Pop_data.columns = Pop_data.columns.str.replace(' ', '')
Pop_data.columns = Pop_data.columns.str.replace('\'','')
Pop_data.rename(columns={'Borough':'persons_sq_mi','County':'persons_sq_km'}, inplace=True)
Pop_data

,NewYorkCitysfiveboroughsvte\n,Jurisdiction\n,Population\n,GrossDomesticProduct\n,Landarea\n,Density\n,persons_sq_mi,squarekm,persons/mi2,persons/km2\n
0,The Bronx\n,\n Bronx\n,"1,418,207\n",42.695\n,"30,100\n",42.10\n,109.04\n,NaN,NaN,NaN
1,Brooklyn\n,\n Kings\n,"2,559,903\n",91.559\n,"35,800\n",70.82\n,183.42\n,NaN,NaN,NaN
2,Manhattan\n,\n New York\n,"1,628,706\n",600.244\n,"368,500\n",22.83\n,59.13\n,NaN,NaN,NaN
3,Queens\n,\n Queens\n,"2,253,858\n",93.310\n,"41,400\n",108.53\n,281.09\n,NaN,NaN,NaN
4,Staten Island\n,\n Richmond\n,"476,143\n",14.514\n,"30,500\n",58.37\n,151.18\n,NaN,NaN,NaN
5,City of New York,"8,336,817",842.343,"101,000",302.64,783.83,"27,547",NaN,NaN,NaN
6,State of New York,"19,453,561","1,731.910","89,000","47,126.40","122,056.82",412,NaN,NaN,NaN
7,Sources:[12][13][14] and see individual boroug...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
Pop_data.drop(columns= {'GrossDomesticProduct\n', 'squarekm', 'persons/mi2', 'persons/km2\n'}, inplace= True)
Pop_data

,NewYorkCitysfiveboroughsvte\n,Jurisdiction\n,Population\n,Landarea\n,Density\n,persons_sq_mi
0,The Bronx\n,\n Bronx\n,"1,418,207\n","30,100\n",42.10\n,109.04\n
1,Brooklyn\n,\n Kings\n,"2,559,903\n","35,800\n",70.82\n,183.42\n
2,Manhattan\n,\n New York\n,"1,628,706\n","368,500\n",22.83\n,59.13\n
3,Queens\n,\n Queens\n,"2,253,858\n","41,400\n",108.53\n,281.09\n
4,Staten Island\n,\n Richmond\n,"476,143\n","30,500\n",58.37\n,151.18\n
5,City of New York,"8,336,817",842.343,302.64,783.83,"27,547"
6,State of New York,"19,453,561","1,731.910","47,126.40","122,056.82",412
7,Sources:[12][13][14] and see individual boroug...,NaN,NaN,NaN,NaN,NaN


In [19]:

Pop_data.rename(columns = {'NewYorkCitysfiveboroughsvte\n' : 'Borough',
                   'Jurisdiction\n':'County',
                   'Population\n':'Estimate_2017', 
                   'Landarea\n':'square_miles',
                    'Density\n':'square_km'}, inplace=True)
Pop_data


,Borough,County,Estimate_2017,square_miles,square_km,persons_sq_mi
0,The Bronx\n,\n Bronx\n,"1,418,207\n","30,100\n",42.10\n,109.04\n
1,Brooklyn\n,\n Kings\n,"2,559,903\n","35,800\n",70.82\n,183.42\n
2,Manhattan\n,\n New York\n,"1,628,706\n","368,500\n",22.83\n,59.13\n
3,Queens\n,\n Queens\n,"2,253,858\n","41,400\n",108.53\n,281.09\n
4,Staten Island\n,\n Richmond\n,"476,143\n","30,500\n",58.37\n,151.18\n
5,City of New York,"8,336,817",842.343,302.64,783.83,"27,547"
6,State of New York,"19,453,561","1,731.910","47,126.40","122,056.82",412
7,Sources:[12][13][14] and see individual boroug...,NaN,NaN,NaN,NaN,NaN


Shift data in the last two rows

In [20]:
Pop_data.loc[5:,['square_km','persons_sq_mi']] = Pop_data.loc[2:,['square_km','persons_sq_mi']].shift(1,axis=1)
Pop_data.loc[5:,['square_miles','square_km']] = Pop_data.loc[2:,['square_miles','square_km']].shift(1,axis=1)
Pop_data.loc[5:,['Estimate_2017','square_miles']] = Pop_data.loc[2:,['Estimate_2017','square_miles']].shift(1,axis=1)
Pop_data.loc[5:,['County','Estimate_2017']] = Pop_data.loc[2:,['County','Estimate_2017']].shift(1,axis=1)
Pop_data.loc[5:,['Borough','County']] = Pop_data.loc[2:,['Borough','County']].shift(1,axis=1)
Pop_data

,Borough,County,Estimate_2017,square_miles,square_km,persons_sq_mi
0,The Bronx\n,\n Bronx\n,"1,418,207\n","30,100\n",42.10\n,109.04\n
1,Brooklyn\n,\n Kings\n,"2,559,903\n","35,800\n",70.82\n,183.42\n
2,Manhattan\n,\n New York\n,"1,628,706\n","368,500\n",22.83\n,59.13\n
3,Queens\n,\n Queens\n,"2,253,858\n","41,400\n",108.53\n,281.09\n
4,Staten Island\n,\n Richmond\n,"476,143\n","30,500\n",58.37\n,151.18\n
5,NaN,City of New York,"8,336,817",842.343,302.64,783.83
6,NaN,State of New York,"19,453,561","1,731.910","47,126.40","122,056.82"
7,NaN,Sources:[12][13][14] and see individual boroug...,NaN,NaN,NaN,NaN


In [21]:
Pop_data['Borough']=Pop_data['Borough'].replace(to_replace='\n', value='', regex=True)
Pop_data['County']=Pop_data['County'].replace(to_replace='\n', value='', regex=True)
Pop_data['Estimate_2017']=Pop_data['Estimate_2017'].replace(to_replace='\n', value='', regex=True)
Pop_data['square_miles']=Pop_data['square_miles'].replace(to_replace='\n', value='', regex=True)
Pop_data['square_km']=Pop_data['square_km'].replace(to_replace='\n', value='', regex=True)
Pop_data['persons_sq_mi']=Pop_data['persons_sq_mi'].replace(to_replace='\n', value='', regex=True)
Pop_data

,Borough,County,Estimate_2017,square_miles,square_km,persons_sq_mi
0,The Bronx,Bronx,"1,418,207","30,100",42.10,109.04
1,Brooklyn,Kings,"2,559,903","35,800",70.82,183.42
2,Manhattan,New York,"1,628,706","368,500",22.83,59.13
3,Queens,Queens,"2,253,858","41,400",108.53,281.09
4,Staten Island,Richmond,"476,143","30,500",58.37,151.18
5,NaN,City of New York,"8,336,817",842.343,302.64,783.83
6,NaN,State of New York,"19,453,561","1,731.910","47,126.40","122,056.82"
7,NaN,Sources:[12][13][14] and see individual boroug...,NaN,NaN,NaN,NaN


Removing NaN and dropping the last row

In [22]:
Pop_data = Pop_data.fillna('')
Pop_data

,Borough,County,Estimate_2017,square_miles,square_km,persons_sq_mi
0,The Bronx,Bronx,"1,418,207","30,100",42.10,109.04
1,Brooklyn,Kings,"2,559,903","35,800",70.82,183.42
2,Manhattan,New York,"1,628,706","368,500",22.83,59.13
3,Queens,Queens,"2,253,858","41,400",108.53,281.09
4,Staten Island,Richmond,"476,143","30,500",58.37,151.18
5,,City of New York,"8,336,817",842.343,302.64,783.83
6,,State of New York,"19,453,561","1,731.910","47,126.40","122,056.82"
7,,Sources:[12][13][14] and see individual boroug...,,,,


In [24]:
Pop_data.drop(7)

,Borough,County,Estimate_2017,square_miles,square_km,persons_sq_mi
0,The Bronx,Bronx,"1,418,207","30,100",42.10,109.04
1,Brooklyn,Kings,"2,559,903","35,800",70.82,183.42
2,Manhattan,New York,"1,628,706","368,500",22.83,59.13
3,Queens,Queens,"2,253,858","41,400",108.53,281.09
4,Staten Island,Richmond,"476,143","30,500",58.37,151.18
5,,City of New York,"8,336,817",842.343,302.64,783.83
6,,State of New York,"19,453,561","1,731.910","47,126.40","122,056.82"


In [25]:
Pop_data.to_csv('BON2_POPULATION.csv',index=False)

#### DEMOGRAPHICS DATA

We will web scrap Demographics data from wikipedia page - https://en.wikipedia.org/wiki/New_York_City using Beautiful soup

In [12]:
# The code was removed by Watson Studio for sharing.

,Racial Composition,2010,1990,1970,1940
0,White,44.00%,52.30%,76.60%,93.60%
1,Non-Hispanic,33.30%,43.20%,62.90%,92%
2,Black or African American,25.50%,28.70%,21.10%,6.10%
3,Hispanic or Latino,28.60%,24.40%,16.20%,1.60%
4,Asian,12.70%,7.00%,1.20%,--


In [13]:
demo_data.to_csv('BON2_DEMOGRAPHICS.csv',index=False)